In [299]:
import pandas as pd
import numpy as np
import calendar

from sklearn.base import BaseEstimator, TransformerMixin


In [336]:
Location = 'Perth'

df = pd.read_csv('../data/weatherAUS.csv')
df = df[df['Location'].isin([Location])]

print(df['RainTomorrow'].value_counts(dropna=False))

df['RainTomorrow'].dropna(axis = 0, inplace = True)
print(df['RainTomorrow'].value_counts(dropna=False))

df['RainTomorrow'] = df['RainTomorrow'].replace(['Yes', 'No'], [1, 0])

df.columns

RainTomorrow
No     2548
Yes     645
Name: count, dtype: int64
RainTomorrow
No     2548
Yes     645
Name: count, dtype: int64


/tmp/ipykernel_19912/61455614.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['RainTomorrow'] = df['RainTomorrow'].replace(['Yes', 'No'], [1, 0])


Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [337]:
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
120638,2008-07-01,Perth,2.7,18.8,0.0,0.8,9.1,ENE,20.0,NaN,...,97.0,53.0,1027.6,1024.5,2.0,3.0,8.5,18.1,No,0
120639,2008-07-02,Perth,6.4,20.7,0.0,1.8,7.0,NE,22.0,ESE,...,80.0,39.0,1024.1,1019.0,0.0,6.0,11.1,19.7,No,0
120640,2008-07-03,Perth,6.5,19.9,0.4,2.2,7.3,NE,31.0,NaN,...,84.0,71.0,1016.8,1015.6,1.0,3.0,12.1,17.7,No,1
120641,2008-07-04,Perth,9.5,19.2,1.8,1.2,4.7,W,26.0,NNE,...,93.0,73.0,1019.3,1018.4,6.0,6.0,13.2,17.7,Yes,1
120642,2008-07-05,Perth,9.5,16.4,1.8,1.4,4.9,WSW,44.0,W,...,69.0,57.0,1020.4,1022.1,7.0,5.0,15.9,16.0,Yes,1


In [338]:
df.isna().sum()

Date               0
Location           0
MinTemp            0
MaxTemp            1
Rainfall           0
Evaporation        1
Sunshine           5
WindGustDir        5
WindGustSpeed      5
WindDir9am       134
WindDir3pm         7
WindSpeed9am       0
WindSpeed3pm       1
Humidity9am        9
Humidity3pm        8
Pressure9am        1
Pressure3pm        1
Cloud9am           2
Cloud3pm           4
Temp9am            0
Temp3pm            1
RainToday          0
RainTomorrow       0
dtype: int64

In [339]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

class VoisinageNAImputer(BaseEstimator, TransformerMixin):
    '''
    Ce transformer remplit les valeurs manquantes dans un DataFrame en utilisant
    une méthode de voisinage basée sur des périodes temporelles autour de chaque NaN.
    Il remplace les NaN dans une colonne par la moyenne des valeurs voisines,
    et gère plusieurs localisations.
    '''

    def __init__(self, column, global_data_path='../data/weatherAUS.csv'):
        '''
        Initialisation avec un DataFrame global lu depuis un fichier CSV (par défaut).
        Le paramètre `column` spécifie la colonne à modifier.
        '''
        self.column = column
        self.global_data_path = global_data_path
        self.global_data = pd.read_csv(self.global_data_path)
    def fit(self, X, y=None):
        '''
        Nous n'avons pas besoin de calculer quoi que ce soit dans fit, sauf valider le DataFrame.
        '''
        self.global_data['Date'] = pd.to_datetime(self.global_data['Date'])
        return self

Il semble que le problème réside dans la manière dont les transformations sont appliquées dans le pipeline, particulièrement avec l'utilisation de VoisinageNAImputer. Voici quelques points à vérifier et quelques suggestions pour que les transformations fonctionnent correctement dans ton pipeline.
Points à vérifier et ajustements :

    Assure-toi que le DataFrame passé à transform() contient bien les colonnes spécifiées :
        Le pipeline fonctionne étape par étape, et si une étape précédente modifie ou ne laisse pas les colonnes requises (par exemple, à cause d'une étape comme drop dans le pipeline), alors le VoisinageNAImputer peut ne pas trouver les colonnes ["Humidity9am", "Humidity3pm", "Pressure9am", "Pressure3pm"] à transformer.
        Il faut t'assurer que ces colonnes ne sont pas supprimées dans le processus du pipeline avant leur passage à VoisinageNAImputer.

    Changer l'indexation de ton DataFrame dans transform() :
        Dans le pipeline, le DataFrame est souvent traité sans tenir compte de l'indexation originale. Tu pourrais essayer de modifier l'indexation dans ta méthode transform pour t'assurer que tout se passe correctement, en évitant des conflits avec les indices après les transformations.

    Pas de modification d'origine dans transform() :
        Tu modifies les colonnes dans transform(), mais cela peut ne pas se refléter dans le pipeline car transform() retourne un nouvel objet. Assure-toi d'effectuer l'affectation directement à la bonne place, dans le même objet.

Suggestions de modifications :

    Vérification et correction de la modification du DataFrame dans transform() : Si la transformation de la colonne dans VoisinageNAImputer ne modifie pas bien les colonnes, voici une approche pour s'assurer que tu modifies les colonnes sans perdre les données :

    def transform(self, X):
        '''
        Applique la logique de remplissage des NaN pour la colonne spécifiée
        en utilisant le global_data stocké dans l'initialisation.
        '''
        X = X.copy()  # Créer une copie pour éviter des problèmes d'assignation dans le pipeline

        X['Date'] = pd.to_datetime(X['Date'])

        # Débogage : vérifier le DataFrame avant transformation
        print("Data avant transformation :")
        print(X.head())

        # Transforme la colonne spécifiée
        X[self.column] = self.remplissage_na_voisinnage(X, column=self.column)

        # Débogage : vérifier le DataFrame après transformation
        print(f"Data après transformation de {self.column}:")
        print(X[self.column].isna().sum())

        return X

    def remplissage_na_voisinnage(self, location, column):
        '''
        Fonction de remplissage des NaN par voisinage. Elle remplace les NaN dans `column` de `location`
        en utilisant la moyenne des voisins autour de la période des NaN, ou la médiane de `global_data` si
        aucun voisin n'est trouvé.
        '''
        i = 0
        location_len = len(location)

        while i < location_len:
            if pd.isna(location[column].iloc[i]):
                periode = 0
                while (i + periode < location_len) and (pd.isna(location[column].iloc[i + periode])):
                    periode += 1

                if periode == location_len:
                    location[column] = self.global_data[column].median()
                    i += location_len
                else:
                    borne_sup = min(location_len - 1, i + periode)
                    date_debut = location["Date"].iloc[i]
                    date_fin = location["Date"].iloc[borne_sup]
                    nombre_jours = (date_fin - date_debut).days + 1

                    voisinnage = location[(location["Date"] >= date_debut - pd.Timedelta(days=nombre_jours)) &
                                          (location["Date"] <= date_fin + pd.Timedelta(days=nombre_jours)) &
                                          (~location["Date"].isna())]

                    if not voisinnage.empty:
                        moyenne_voisins = voisinnage[column].mean()
                        location.loc[location.index[i]:location.index[borne_sup], column] = moyenne_voisins
                    else:
                        location[column] = self.global_data[column].median()
                    i += periode
            else:
                i += 1

        return location[column]

In [ ]:
    ["Humidity9am", "Humidity3pm", "Pressure9am", "Pressure3pm"]

essai = VoisinageNAImputer(column = "Pressure9am").fit_transform(df)
essai.isna().sum()

Date               0
Location           0
MinTemp            0
MaxTemp            1
Rainfall           0
Evaporation        1
Sunshine           5
WindGustDir        5
WindGustSpeed      5
WindDir9am       134
WindDir3pm         7
WindSpeed9am       0
WindSpeed3pm       1
Humidity9am        9
Humidity3pm        8
Pressure9am        0
Pressure3pm        1
Cloud9am           2
Cloud3pm           4
Temp9am            0
Temp3pm            1
RainToday          0
RainTomorrow       0
dtype: int64